In [1]:
#from __future__ import print_function
import argparse
import os
import imp
import algorithms as alg
from dataloader import DataLoader, GenericDataset

In [2]:
import torch
import torch.nn as nn
torch.cuda.device_count()

1L

In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [4]:
args_opt = Namespace(
    exp = 'SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features',
    evaluate = False,
    checkpoint = 0,
    num_workers = 1,
    cuda = True,
    disp_step= 5
)

In [5]:
exp_config_file = os.path.join('.','config',args_opt.exp+'.py')
# if args_opt.semi == -1:
exp_directory = os.path.join('.','experiments',args_opt.exp)
# else:
#    assert(args_opt.semi>0)
#    exp_directory = os.path.join('.','experiments/unsupervised',args_opt.exp+'_semi'+str(args_opt.semi))

In [6]:
exp_config_file

'./config/SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features.py'

In [7]:
# Load the configuration params of the experiment
print('Launching experiment: %s' % exp_config_file)
config = imp.load_source("",exp_config_file).config
config['exp_dir'] = exp_directory # the place where logs, models, and other stuff will be stored
print("Loading experiment %s from file: %s" % (args_opt.exp, exp_config_file))
print("Generated logs, snapshots, and model files will be stored on %s" % (config['exp_dir']))

Launching experiment: ./config/SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features.py
Loading experiment SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features from file: ./config/SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features.py
Generated logs, snapshots, and model files will be stored on ./experiments/SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features


In [8]:
# Set train and test datasets and the corresponding data loaders
data_train_opt = config['data_train_opt']
data_test_opt = config['data_test_opt']
num_imgs_per_cat = data_train_opt['num_imgs_per_cat'] if ('num_imgs_per_cat' in data_train_opt) else None

In [9]:
dataset_train = GenericDataset(
    dataset_name=data_train_opt['dataset_name'],
    split=data_train_opt['split'],
    random_sized_crop=data_train_opt['random_sized_crop'],
num_imgs_per_cat=num_imgs_per_cat)

custom


In [10]:
dataset_test = GenericDataset(
    dataset_name=data_test_opt['dataset_name'],
    split=data_test_opt['split'],
    random_sized_crop=data_test_opt['random_sized_crop'])

custom


In [11]:
dloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=data_train_opt['batch_size'],
    unsupervised=data_train_opt['unsupervised'],
    epoch_size=data_train_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=True)

In [12]:
dloader_test = DataLoader(
    dataset=dataset_test,
    batch_size=data_test_opt['batch_size'],
    unsupervised=data_test_opt['unsupervised'],
    epoch_size=data_test_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=False)

In [13]:
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms

# trainDataDir = '/scratch/um367/DL/data/sampledata/supervised/train'
# valDataDir = '/scratch/um367/DL/data/sampledata/supervised/val'

# kwargs = {'num_workers': 1, 'pin_memory': True} if args_opt.cuda else {}

# dloader_train = torch.utils.data.DataLoader(
#     datasets.ImageFolder(trainDataDir, transform=transforms.ToTensor()),
#     batch_size=data_train_opt['batch_size'], shuffle=True, **kwargs)

# dloader_test = torch.utils.data.DataLoader(
#     datasets.ImageFolder(valDataDir, transform=transforms.ToTensor()),
#     batch_size=data_test_opt['batch_size'], shuffle=True, **kwargs)

In [14]:
len(dloader_train.dataset)

64000

In [15]:
config['disp_step'] = args_opt.disp_step

In [16]:
algorithm = getattr(alg, config['algorithm_type'])(config)

2019-05-15 02:07:32,267 - algorithms.Algorithm - INFO   - Algorithm options {'max_num_epochs': 40, 'disp_step': 5, 'algorithm_type': 'FeatureClassificationModel', 'criterions': {'loss': {'opt': None, 'ctype': 'CrossEntropyLoss'}}, 'data_test_opt': {'unsupervised': False, 'batch_size': 150, 'epoch_size': None, 'random_sized_crop': False, 'split': 'supervised/val', 'dataset_name': 'custom'}, 'exp_dir': './experiments/SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features', 'out_feat_keys': ['conv5'], 'networks': {'classifier': {'opt': [{'cls_type': 'Vae', 'num_classes': 1000, 'nChannels': 256}], 'def_file': 'architectures/MultipleNonLinearClassifiers.py', 'optim_params': {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9}, 'pretrained': None}, 'feat_extractor': {'opt': {'num_classes': 4}, 'def_file': 'architectures/vaecnn.py', 'optim_params': None, 'pretrained': './vaeSiam.pth.t

init_all_networks()
[('classifier', {'opt': [{'cls_type': 'Vae', 'num_classes': 1000, 'nChannels': 256}], 'def_file': 'architectures/MultipleNonLinearClassifiers.py', 'optim_params': {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9}, 'pretrained': None}), ('feat_extractor', {'opt': {'num_classes': 4}, 'def_file': 'architectures/vaecnn.py', 'optim_params': None, 'pretrained': './vaeSiam.pth.tar'})]


2019-05-15 02:07:34,581 - algorithms.Algorithm - INFO   - Set network feat_extractor
2019-05-15 02:07:34,583 - algorithms.Algorithm - INFO   - ==> Initiliaze network feat_extractor from file architectures/vaecnn.py with opts: {'num_classes': 4}
2019-05-15 02:07:34,976 - algorithms.Algorithm - INFO   - ==> Load pretrained parameters from file ./vaeSiam.pth.tar:
2019-05-15 02:07:55,351 - algorithms.Algorithm - INFO   - Initialize criterion[loss]: CrossEntropyLoss with options: None


init_all_criterions()
allocate_tensors()


In [17]:
# device = torch.device("cuda")
# device
256*6*6

9216

In [18]:
if args_opt.cuda: # enable cuda
#     if torch.cuda.device_count() > 1:
#         algorithm = nn.DataParallel(algorithm)
    algorithm.load_to_gpu()
#     algorithm.to(device)
if args_opt.checkpoint > 0: # load checkpoint
    algorithm.load_checkpoint(args_opt.checkpoint, train= (not args_opt.evaluate))

In [ ]:
if not args_opt.evaluate: # train the algorithm
    algorithm.solve(dloader_train, dloader_test)
else:
    algorithm.evaluate(dloader_test) # evaluate the algorithm


2019-05-15 02:07:55,883 - algorithms.Algorithm - INFO   - Initialize optimizer: sgd with params: {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9} for netwotk: classifier
2019-05-15 02:07:55,886 - algorithms.Algorithm - INFO   - Training epoch [  1 /  40]
2019-05-15 02:07:55,887 - algorithms.Algorithm - INFO   - ==> Set to classifier optimizer lr = 0.0100000000
2019-05-15 02:07:55,889 - algorithms.Algorithm - INFO   - Training: SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
 37%|███▋      | 159/427 [08:28<14:50,  3.32s/it]2019-05-15 02:16:27,859 - algorithms.Algorithm - INFO   - ==> Iteration [  1][ 160 /  426]: {'process_time': 85.4099, 'prec5_c1': 2.0417, 'loss': 6.9764, 'prec1_c1': 0.4833, 'load_time': 14.5901}


 77%|███████▋  | 329/427 [17:12<04:57,  3.04s/it]2019-05-15 02:25:11,978 - algorithms.Algorithm - INFO   - ==> Iteration [  1][ 330 /  426]: {'process_time': 85.7574, 'prec5_c1': 2.8303, 'loss': 6.8137, 'prec1_c1': 0.7313, 'load_time': 14.2426}


100%|██████████| 427/427 [22:15<00:00,  2.88s/it]
2019-05-15 02:30:11,762 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 85.6748, 'prec5_c1': 3.1062, 'loss': 6.7576, 'prec1_c1': 0.8228, 'load_time': 14.3252}
2019-05-15 02:30:12,828 - algorithms.Algorithm - INFO   - Evaluating: SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
2019-05-15 02:30:12,830 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
100%|██████████| 427/427 [16:32<00:00,  2.01s/it]
2019-05-15 02:46:45,411 - algorithms.Algorithm - INFO   - ==> Results: {'process_time': 83.1837, 'prec5_c1': 5.2498, 'loss': 6.4867, 'prec1_c1': 1.6073, 'load_time': 16.8163}
2019-05-15 02:46:45,486 - algorithms.Algorithm - INFO   - ==> Evaluation stats: {'process_time': 83.1837, 'prec5_c1': 5.2498, 'loss': 6.4867, 'prec1_c1': 1.6073, 'load_time': 16.8163}
2019-05-15 02:46:45,488 - algorithms.Algorithm - INFO   - Training epoch [  2 /  40]
2019-05-15 02:46:45,490 - algorith

 51%|█████▏    | 219/427 [10:54<11:19,  3.27s/it]2019-05-15 02:57:43,054 - algorithms.Algorithm - INFO   - ==> Iteration [  2][ 220 /  426]: {'process_time': 86.5135, 'prec5_c1': 5.3939, 'loss': 6.4201, 'prec1_c1': 1.6091, 'load_time': 13.4865}


 91%|█████████ | 389/427 [20:21<01:57,  3.09s/it]2019-05-15 03:07:09,260 - algorithms.Algorithm - INFO   - ==> Iteration [  2][ 390 /  426]: {'process_time': 85.3135, 'prec5_c1': 5.5231, 'loss': 6.4099, 'prec1_c1': 1.6855, 'load_time': 14.6865}


...

 56%|█████▌    | 239/427 [12:35<09:07,  2.91s/it]2019-05-15 07:09:09,536 - algorithms.Algorithm - INFO   - ==> Iteration [  8][ 240 /  426]: {'process_time': 84.2675, 'prec5_c1': 11.525, 'loss': 5.8886, 'prec1_c1': 3.9389, 'load_time': 15.7325}


 96%|█████████▌| 409/427 [21:15<00:59,  3.33s/it]2019-05-15 07:17:49,682 - algorithms.Algorithm - INFO   - ==> Iteration [  8][ 410 /  426]: {'process_time': 84.2535, 'prec5_c1': 11.3837, 'loss': 5.9039, 'prec1_c1': 3.8537, 'load_time': 15.7465}


100%|██████████| 427/427 [22:12<00:00,  2.87s/it]
2019-05-15 07:18:43,947 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 84.4418, 'prec5_c1': 11.3763, 'loss': 5.9041, 'prec1_c1': 3.8462, 'load_time': 15.5582}
2019-05-15 07:18:44,828 - algorithms.Algorithm - INFO   - Evaluating: SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
2019-05-15 07:18:44,851 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
100%|██████████| 427/427 [17:10<00:00,  1.86s/it]
2019-05-15 07:35:54,883 - algorithms.Algorithm - INFO   - ==> Results: {'process_time': 82.8719, 'prec5_c1': 10.3755, 'loss': 6.0897, 'prec1_c1': 3.6409, 'load_time': 17.1281}
2019-05-15 07:35:54,909 - algorithms.Algorithm - INFO   - ==> Evaluation stats: {'process_time': 82.8719, 'prec5_c1': 10.3755, 'loss': 6.0897, 'prec1_c1': 3.6409, 'load_time': 17.1281}
2019-05-15 07:35:54,911 - algorithms.Algorithm - INFO   - Training epoch [  9 /  40]
2019-05-15 07:35:54,912 - algor

 70%|███████   | 299/427 [15:31<06:29,  3.04s/it]2019-05-15 07:51:29,452 - algorithms.Algorithm - INFO   - ==> Iteration [  9][ 300 /  426]: {'process_time': 85.074, 'prec5_c1': 11.8467, 'loss': 5.8474, 'prec1_c1': 4.08, 'load_time': 14.926}


100%|██████████| 427/427 [23:07<00:00,  3.34s/it]
2019-05-15 07:59:02,386 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 85.4738, 'prec5_c1': 11.7963, 'loss': 5.8555, 'prec1_c1': 4.057, 'load_time': 14.5262}
2019-05-15 07:59:03,263 - algorithms.Algorithm - INFO   - Evaluating: SiamVae_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features
2019-05-15 07:59:03,265 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
 27%|██▋       | 116/427 [06:12<16:16,  3.14s/it]

/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torchvision/transforms/functional.py", line 48, in to_tensor